Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import random
%matplotlib inline

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Machine Learning Colab/Shopee/shopee-product-matching/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Machine Learning Colab/Shopee/shopee-product-matching/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/Machine Learning Colab/Shopee/shopee-product-matching/sample_submission.csv')

In [3]:
train.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


Data Info

In [4]:
vc = dict(train.label_group.value_counts())
counts = list(set(vc.values()))
print('min = ', min(counts), ", max = ", max(counts), ', all = ', sum(vc.values()))

min =  2 , max =  51 , all =  34250


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [6]:
for i in train.columns:
    print(i, len(np.unique(train[i])))

posting_id 34250
image 32412
image_phash 28735
title 33117
label_group 11014


In [8]:
train=train.drop(['image_phash','title'], axis=1)
train.head()

,posting_id,image,label_group,image_path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,249114794,/content/drive/MyDrive/Machine Learning Colab/...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,2937985045,/content/drive/MyDrive/Machine Learning Colab/...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,2395904891,/content/drive/MyDrive/Machine Learning Colab/...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,4093212188,/content/drive/MyDrive/Machine Learning Colab/...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,3648931069,/content/drive/MyDrive/Machine Learning Colab/...


Data Path

In [9]:
DATA_PATH = '/content/drive/MyDrive/Machine Learning Colab/Shopee/shopee-product-matching'
train['image_path'] = DATA_PATH + 'train_images/' + train['image']
train.head()

,posting_id,image,label_group,image_path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,249114794,/content/drive/MyDrive/Machine Learning Colab/...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,2937985045,/content/drive/MyDrive/Machine Learning Colab/...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,2395904891,/content/drive/MyDrive/Machine Learning Colab/...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,4093212188,/content/drive/MyDrive/Machine Learning Colab/...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,3648931069,/content/drive/MyDrive/Machine Learning Colab/...


In [8]:
dftrain=train.drop(['posting_id','image_path'], axis=1)
dftrain.head()

,image,image_phash,title,label_group
0,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [10]:
dftest=test.drop(['posting_id','image_phash','title'], axis=1)
dftest.head()

,image
0,0006c8e5462ae52167402bac1c2e916e.jpg
1,0007585c4d0f932859339129f709bfdc.jpg
2,0008377d3662e83ef44e1881af38b879.jpg


In [11]:
image_path = '/content/drive/MyDrive/Machine Learning Colab/Shopee/shopee-product-matching/train_images'

import cv2

img = cv2.imread(image_path)

# if pad:
#     pad_width = get_pad_width(img, max(img.shape))
#     padded = np.pad(img, pad_width=pad_width, mode='constant', constant_values=0)
# else:
#     padded = img

# padded = cv2.cvtColor(padded, cv2.COLOR_BGR2RGB)
# resized = cv2.resize(padded, (desired_size,)*2).astype('uint8')

# return resized


Image CNN

In [12]:
from PIL import Image

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

# d = ShopeeImageDataset()
# d[10]
# len[10]
class ShopeeImageDataset(Dataset):
    def __init__(self, img_path, transform):
        self.img_path = img_path
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        img = self.transform(img)
        return img
    
    def __len__(self):
        return len(self.img_path)

In [13]:
imagedataset = ShopeeImageDataset(
    train['image'].values,
    transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))


imageloader = torch.utils.data.DataLoader(
    imagedataset,
    batch_size=10, shuffle=False, num_workers=2
)

In [14]:
class ShopeeImageEmbeddingNet(nn.Module):
    def __init__(self):
        super(ShopeeImageEmbeddingNet, self).__init__()
              
        model = models.resnet18(True)  
        model.avgpool = nn.AdaptiveMaxPool2d(output_size=(1, 1))
        
        
        model = nn.Sequential(*list(model.children())[:-1])
        model.eval() 
        self.model = model
        
    def forward(self, img):        
        out = self.model(img)
        return out

In [15]:
imgmodel = ShopeeImageEmbeddingNet()
imgmodel = imgmodel.to()

# imagefeat = []
# with torch.no_grad(): 
#     for data in tqdm_notebook(imageloader):
#         data = data.to()
#         feat = imgmodel(data)
#         feat = feat.reshape(feat.shape[0], feat.shape[1])
#         feat = feat.data.cpu().numpy()
        
#         imagefeat.append(feat)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Normalize

정규화
- 머신러닝의 핵심은 cost를 구하고 Gradient Descent 와 같은 알고리즘을 사용해 cost를 최소화 시키는 것. 그렇기 위해 적당한 Learning Rate 값을 찾아야 함.
- 정규화되지 않은 데이터셋은 Learning Rate를 작게 해야만 학습이 될 수 있음. 충분히 작지 않다면 불균형이 발생.
- 정규화된 데이터셋은 Gradient Descent 알고리즘을 적용해 쉽고 빠르게 최적화 지점을 찾을 수 있음.

Data Normalization의 방법
1. Normalization (Min/Max)
전체 구간을 [0,1]로 맞춤.
2. Standardization
original data : 정규화 하기 전의 데이터 분포
zero-centered data : 원 데이터에 평균을 뺀 것. 데이터의 분포가 가운데에 모이게 됨.
normalized data : 표준편차를 나눠주면서 데이터의 분포가 일정해지는 효과. (가로=세로)

반드시 정규화를 하기 위해 평균, 표준편차, min, max 등 값들은 train set에 있는 값만 사용해 구하고, validation,test set에 적용해야함.



In [16]:
!pip install arr

     |████████████████████████████████| 245kB 4.4MB/s 
     |████████████████████████████████| 22.9MB 1.5MB/s 
     |████████████████████████████████| 6.7MB 29.8MB/s 
     |████████████████████████████████| 6.5MB 26.3MB/s 
     |████████████████████████████████| 22.3MB 1.8MB/s 
     |████████████████████████████████| 256kB 46.7MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 409kB 49.9MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 839kB 40.1MB/s 
     |████████████████████████████████| 110.5MB 89kB/s 
     |████████████████████████████████| 512kB 23.4MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 3.8MB 29.8MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 3.2MB 44.7MB/s 
  Created wheel for tess-point: filename=tess_point-0.6.1-cp37-none-any.whl size=22449 sha256=fcc7e168dcf88f6fc85

In [18]:
from sklearn.preprocessing import normalize

# l2 norm to kill all the sim in 0-1
# 3.8w * 512

# imagefeat = np.vstack(imagefeat)
import arr
arr = arr.reshape(-1, 1)
imagefeat = normalize(imagefeat)


AttributeError: ignored

In [20]:
# KNN = 50
# if len(test)==3: KNN = 2
# model = NearestNeighbors(n_neighbors=KNN)
# model.fit(imagefeat)

In [23]:
# preds = []
# CHUNK = 1024*4

# imagefeat = cupy.array(imagefeat)

# print('Finding similar images...')
# CTS = len(imagefeat)//CHUNK
# if len(imagefeat)%CHUNK!=0: CTS += 1
# for j in range( CTS ):
    
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b, len(imagefeat))
#     print('chunk',a,'to',b)
    
#     # GPU
#     distances = cupy.matmul(imagefeat, imagefeat[a:b].T).T
#     # distances = np.dot(imagefeat[a:b,], imagefeat.T)
    
#     for k in range(b-a):
#         IDX = cupy.where(distances[k,]>0.95)[0]
#         # IDX = np.where(distances[k,]>0.95)[0][:]
#         o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
#         preds.append(o)
        
# # del imagefeat, imgmodel

In [22]:

# if COMPUTE_CV:
#     train['f1'] = train.apply(getMetric('oof_cnn'),axis=1)
#     print('CV score for baseline =',train.f1.mean())

In [25]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(lr=0.00002),
    metrics=['accuracy']
)

return model

NameError: ignored

Load Labels

In [ ]:
train_df = dftrain test_df = dftest

Train Model

In [19]:

from __future__ import print_function
import keras
from keras.models import Sequential



history = model1.fit(train_data, train_labels_one_hot, batch_size=batch_size, epochs=epochs, verbose=1, 
                   validation_data=(test_data, test_labels_one_hot))
model1.evaluate(test_data, test_labels_one_hot)

plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)


plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

NameError: ignored

In [1]:
# Submission.csv
print(test_df)

NameError: ignored